# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weatherdf = pd.read_csv('../WeatherPy/clean_data.csv')
weatherdf.head()

,City,Country,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude
0,Vaini,TO,68.00,90,94,6.93,-21.20,-175.20
1,Kota Bharu,MY,78.80,20,88,2.24,6.13,102.24
2,George Town,MY,80.60,40,100,5.70,5.41,100.34
3,Bluff,NZ,39.00,89,83,3.00,-46.60,168.33
4,Albany,US,96.01,63,41,9.08,42.60,-73.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
gmaps.configure(g_key)
coordinates = weatherdf[["Latitude", "Longitude"]].astype(float)
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=(weatherdf["Humidity"]), 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
plt.savefig("vacaheatmap.png")
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

###### 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
lowwind1 = weatherdf.loc[weatherdf["Wind Speed"]<10,:]
nocloud2 = lowwind1.loc[lowwind1["Cloudiness"]==0,:]
nicetemp3 = nocloud2.loc[nocloud2["Max Temperature"]<80,:]
final4 = nicetemp3.loc[nicetemp3['Max Temperature']>70,:]

final4.head()




,City,Country,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude
92,Sirte,LY,77.05,0,75,9.71,31.21,16.59
137,Casablanca,MA,75.00,0,94,8.05,33.59,-7.62
170,Mingora,PK,76.60,0,58,5.39,34.78,72.36
187,Nador,MA,77.00,0,78,3.36,35.17,-2.93
201,Vila,PT,73.99,0,82,6.93,42.03,-8.16


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(final4)
hotel_df['Hotel'] = ""
hotel_df

,City,Country,Max Temperature,Cloudiness,Humidity,Wind Speed,Latitude,Longitude,Hotel
92,Sirte,LY,77.05,0,75,9.71,31.21,16.59,
137,Casablanca,MA,75.00,0,94,8.05,33.59,-7.62,
170,Mingora,PK,76.60,0,58,5.39,34.78,72.36,
187,Nador,MA,77.00,0,78,3.36,35.17,-2.93,
201,Vila,PT,73.99,0,82,6.93,42.03,-8.16,
256,Erenhot,CN,70.20,0,32,7.36,43.65,111.98,
295,Porto Torres,IT,71.60,0,88,2.24,40.84,8.40,
332,Kumluca,TR,79.23,0,60,3.13,36.37,30.29,
350,Henties Bay,NaN,71.01,0,24,7.43,-22.12,14.28,
396,Tabuk,PH,72.81,0,86,4.09,17.42,121.44,


In [10]:
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']
    
    params["location"] = f"{latitude},{longitude}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][0]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        hotel_df.loc[index, "Hotel"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing... skipping.")
        
    print("------------")

Retrieving Results for Index 92: Sirte.
Closest hotel to Sirte at 31.21 , 16.59 is فندق الدويهش.
------------
Retrieving Results for Index 137: Casablanca.
Closest hotel to Casablanca at 33.59 , -7.62 is Four Seasons Hotel Casablanca.
------------
Retrieving Results for Index 170: Mingora.
Closest hotel to Mingora at 34.78 , 72.36 is Pearl Continental Hotel Malam Jabba.
------------
Retrieving Results for Index 187: Nador.
Closest hotel to Nador at 35.17 , -2.93 is Hotel Marchica Lagoon Resort, Nador Morocco.
------------
Retrieving Results for Index 201: Vila.
Closest hotel to Vila at 42.03 , -8.16 is Tempus Hotel & Spa.
------------
Retrieving Results for Index 256: Erenhot.
Closest hotel to Erenhot at 43.65 , 111.98 is Moli International Hotel.
------------
Retrieving Results for Index 295: Porto Torres.
Closest hotel to Porto Torres at 40.84 , 8.4 is Hotel & SPA Riviera Castelsardo.
------------
Retrieving Results for Index 332: Kumluca.
Closest hotel to Kumluca at 36.37 , 30.29 is

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [25]:
hotel_df.head()

In [29]:
## Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights= (hotel_df["Humidity"]), 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("markermapvaca.png")

fig



Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>